In [ ]:
!pip install underthesea
!pip install Unidecode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
!unzip "/content/drive/MyDrive/DL_Final/dataset.zip"

In [ ]:
import torch
from underthesea import sent_tokenize
from collections import Counter
from underthesea import word_tokenize
from torch.utils.data import Dataset
from pathlib import Path
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import unidecode
import re
import random
import torch.nn.functional as F
import itertools

In [ ]:
data_path = "corpus-title.txt"
import pickle
# Save vocabulary to a file
def save_vocab(vocab, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(vocab, file)

# Load vocabulary from a file
def load_vocab(file_path):
    with open(file_path, 'rb') as file:
        vocab = pickle.load(file)
    return vocab
vocab_file_path = "vocab.pkl"
checkpoint_file_path = "/content/drive/MyDrive/DL_Final/seq2seq_checkpoint_epoch_5.pt"

In [ ]:
class Vocabulary:
    def __init__(self,freq_threshold):
        #setting the pre-reserved tokens int to string tokens
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}

        #string to int tokens
        #its reverse dict self.itos
        self.stoi = {v:k for k,v in self.itos.items()}

        self.freq_threshold = freq_threshold

    def __len__(self): return len(self.itos)

    @staticmethod
    def tokenize(text):
        return [token.lower() for token in word_tokenize(text)]

    def build_vocab(self, sentence_list):
        frequencies = Counter()
        idx = 4
        for sentence in tqdm(sentence_list):
            for word in self.tokenize(sentence):
                frequencies[word] += 1
                #add the word to the vocab if it reaches minum frequecy threshold
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self,text):
        tokenized_text = self.tokenize(text)
        return [ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text ]

    def get_pad_index(self):
        return self.stoi["<PAD>"]

    def get_start_index(self):
        return self.stoi["<SOS>"]

    def get_end_index(self):
        return self.stoi["<EOS>"]

In [ ]:
accented_chars_vietnamese = [
    'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ',
    'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ',
    'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
    'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự',
    'í', 'ì', 'ỉ', 'ĩ', 'ị',
    'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ',
    'đ',
]

accented_chars_vietnamese.extend([c.upper() for c in accented_chars_vietnamese])
alphabet = list("aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0123456789! \"#$%&''()*+,-./:;<=>?@[\]^_{|}~")

chars_regrex = '[aàảãáạăằẳẵắặâầẩẫấậAÀẢÃÁẠĂẰẲẴẮẶÂẦẨẪẤẬoòỏõóọôồổỗốộơờởỡớợOÒỎÕÓỌÔỒỔỖỐỘƠỜỞỠỚỢeèẻẽéẹêềểễếệEÈẺẼÉẸÊỀỂỄẾỆuùủũúụưừửữứựUÙỦŨÚỤƯỪỬỮỨỰiìỉĩíịIÌỈĨÍỊyỳỷỹýỵYỲỶỸÝỴnNvVmMCG]'
same_chars = {
    'a': ['á', 'à', 'ả', 'ã', 'ạ', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ'],
    'A': ['Á','À','Ả','Ã','Ạ','Ấ','Ầ','Ẩ','Ẫ','Ậ','Ắ','Ằ','Ẳ','Ẵ','Ặ'],
    'O': ['Ó','Ò','Ỏ','Õ','Ọ','Ô','Ố','Ồ','Ổ','Ỗ','Ộ','Ơ','Ớ','Ờ','Ở','Ỡ','Ợ','Q'],
    'o': ['ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ','ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'q'],
    'e': ['é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ê'],
    'E': ['É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ê'],
    'u': ['ú', 'ù', 'ủ', 'ũ', 'ụ', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'ư'],
    'U': ['Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Ư'],
    'i': ['í', 'ì', 'ỉ', 'ĩ', 'ị'],
    'I': ['Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị'],
    'y': ['ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'v'],
    'Y': ['Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'V'],
    'n': ['m'],
    'N': ['N'],
    'v': ['y'],
    'V': ['Y'],
    'm': ['n'],
    'M': ['N'],
    'C': ['G'],
    'G': ['C']
}

indices = [alphabet.index(c) + 4 for c in accented_chars_vietnamese]
vocab_size = len(alphabet)
litle_char = list('"#$%&''()*+,-./:;<=>?@[\]^_{|}~')

In [ ]:
def remove_random_accent(text, ratio=0.15):
        text = ''.join(text)
        words = text.split()
        mask = np.random.random(size=len(words)) < ratio

        for i in range(len(words)):
            if mask[i]:
                words[i] = unidecode.unidecode(words[i])
                break

        return ' '.join(words)

def remove_random_space(text):
        text = ''.join(text)
        words = text.split()
        n_words = len(words)
        start = np.random.randint(low=0, high=n_words, size=1)[0]

        if start + 3 < n_words:
            end = np.random.randint(low=start, high=start + 3, size=1)[0]
        else:
            end = np.random.randint(low=start, high=n_words, size=1)[0]

        out = ' '.join(words[:start])  + ' ' + ''.join(words[start:end + 1]) + ' ' + ' '.join(words[end + 1:])

        return out.strip()

def _char_regrex(text):
        match_chars = re.findall(chars_regrex, text)

        return match_chars

def _random_replace(text, match_chars):
        replace_char = match_chars[np.random.randint(low=0, high=len(match_chars), size=1)[0]]
        insert_chars = same_chars[unidecode.unidecode(replace_char)]
        insert_char = insert_chars[np.random.randint(low=0, high=len(insert_chars), size=1)[0]]
        text = text.replace(replace_char, insert_char, 1)

        return text

def change(text):
        match_chars = _char_regrex(text)
        if len(match_chars) == 0:
            return text
        text = ''.join(text)
        text = _random_replace(text, match_chars)

        return text

def replace_accent_chars(text, ratio=0.15):
        text = ''.join(text)
        words = text.split()
        mask = np.random.random(size=len(words)) < ratio

        for i in range(len(words)):
            if mask[i]:
                words[i] = change(words[i])
                break

        return ' '.join(words)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_path, vocab):
        self.data = self.load_data(data_path)
        self.vocab = vocab


    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
      function_list = [remove_random_accent, replace_accent_chars,remove_random_space]
      random_function = random.choice(function_list)
      target_text = ''.join(self.data[idx])
      source_text = random_function(source_text)
      numericalized_source = self.vocab.numericalize(source_text)
      numericalized_target = self.vocab.numericalize(target_text)

      # Pad or truncate source and target sequences to a fixed length
      max_length = 20  # Define your desired maximum sequence length
      numericalized_source = self.pad_or_truncate(numericalized_source, max_length)
      numericalized_target = self.pad_or_truncate(numericalized_target, max_length)
      return torch.tensor(numericalized_source), torch.tensor(numericalized_target, dtype=torch.long)

    def pad_or_truncate(self, sequence, max_length):
      if len(sequence) < max_length:
          # Pad sequence with padding token
          padding_token = str(self.vocab.get_pad_index())
          sequence += [padding_token] * (max_length - len(sequence))
      elif len(sequence) > max_length:
          # Truncate sequence to max_length
          sequence = sequence[:max_length]
      sequence = [ int(x) for x in sequence ]
      return sequence

    def load_data(self, data_path):
        # Load data from the file
        with open(data_path, 'r', encoding='utf-8') as file:
            #lines = [line.strip().split('\t') for line in file.readlines()]
            lines = [file.readline().strip() for _ in range(200000)]
            print("Reading dataset with size: " + str(len(lines)))
        return lines


In [ ]:
freq_threshold = 5
if not Path(vocab_file_path).is_file():
  vocab = Vocabulary(freq_threshold)
  dataset = CustomDataset(data_path, vocab)
  vocab.build_vocab(dataset.data)
else:
  vocab = Vocabulary(freq_threshold)
  vocab = load_vocab(vocab_file_path)

In [ ]:
save_vocab(vocab, vocab_file_path)

In [ ]:
def collate_batch(batch):
    """
    Collate function for DataLoader to pad sequences.

    Args:
    batch (list): List of tuples, each containing a source sequence and a target sequence.

    Returns:
    tuple: A tuple containing padded source sequences and padded target sequences.
    """
    # Unzip the batch into source and target sequences
    src_batch, trg_batch = zip(*batch)

    # Find the lengths of each source and target sequence
    src_lengths = [len(seq) for seq in src_batch]
    trg_lengths = [len(seq) for seq in trg_batch]

    # Find the maximum length in each batch
    max_src_length = max(src_lengths)
    max_trg_length = max(trg_lengths)

    # Get the padding index
    pad_index = vocab.get_pad_index()

    # Pad the source sequences
    src_padded = torch.nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=pad_index)

    # Pad the target sequences
    trg_padded = torch.nn.utils.rnn.pad_sequence(trg_batch, batch_first=True, padding_value=pad_index)

    return src_padded, trg_padded


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_dim, num_layers)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

# Define your Decoder
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_dim, num_layers)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

# Define your Seq2Seq model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.fc_out.out_features

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(src.device)
        hidden, cell = self.encoder(src)

        input = trg[0, :]

        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs


In [ ]:
# Define parameters
input_dim = len(vocab)
output_dim = len(vocab)
emb_dim = 256
hidden_dim = 512
num_layers = 2
batch_size = 64
num_epochs = 5
learning_rate = 0.001
checkpoint_path = "/content/drive/MyDrive/DL_Final/seq2seq_checkpoint_epoch_{}.pt"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create an instance of the encoder and decoder
encoder = Encoder(input_dim, emb_dim, hidden_dim, num_layers)
decoder = Decoder(output_dim, emb_dim, hidden_dim, num_layers)

# Create an instance of the Seq2Seq model
model = Seq2Seq(encoder, decoder).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader for training
train_dataset = CustomDataset(data_path, vocab)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)

if not Path(checkpoint_file_path).is_file():
  # Training loop
  for epoch in range(num_epochs):
      total_loss = 0
      for src, trg in tqdm(train_loader):
          src = src.to(device)
          trg = trg.to(device)

          optimizer.zero_grad()
          output = model(src, trg)

          # Shape of output: (trg_len, batch_size, output_dim)
          # Shape of trg: (trg_len, batch_size)
          output_dim = output.shape[-1]

          output = output[1:].view(-1, output_dim)
          trg = trg[1:].view(-1)

          loss = criterion(output, trg)

          loss.backward()
          optimizer.step()

          total_loss += loss.item()

      print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")
      avg_loss = total_loss / len(train_loader)
      checkpoint_file = checkpoint_path.format(epoch + 1)
      torch.save({
          'epoch': epoch + 1,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': avg_loss,
          # Add any other relevant information
      }, checkpoint_file)
else:
  checkpoint = torch.load(checkpoint_file_path, map_location=torch.device('cpu'))
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  start_epoch = checkpoint['epoch']

In [ ]:
def translate_sentence(sentence, model, device, max_length=50, sos_token_idx=1, eos_token_idx=2):
    model.eval()

    with torch.no_grad():
        # Tokenize the input sentence
        numericalized_sentence = [vocab.numericalize(token) for token in sentence.split()]
        numericalized_sentence = list(itertools.chain.from_iterable(numericalized_sentence))
        numericalized_sentence = torch.tensor(numericalized_sentence).unsqueeze(1).to(device)  # Modified here
       # numericalized_sentence = torch.tensor(numericalized_sentence).to(device)

        # Forward pass through the model
        hidden, cell = model.encoder(numericalized_sentence)
        output_sequence = [sos_token_idx]

        for _ in range(max_length):
            input_tensor = torch.tensor([output_sequence[-1]]).to(device)
            output, hidden, cell = model.decoder(input_tensor, hidden, cell)
            output_token = output.argmax(1).item()
            output_sequence.append(output_token)
            if output_token == eos_token_idx:
                break

        # Convert output tokens to text
        output_sentence = [vocab.itos[idx] for idx in output_sequence[1:-1]]  # Exclude <sos> and <eos> tokens
        return ' '.join(output_sentence)

# Example usage:
input_sentence = "Bạn có thấy khỏe không?"
output_sentence = translate_sentence(input_sentence, model, device)
print(output_sentence)
